In [254]:
import numpy as np
import pandas as pd

In [255]:
###################### Funcion calcular sindrome ######################
def calcular_sindrome(patron_error, matriz_Ht):
    sindrome = []
    detectable = []
    for i in patron_error:
        valor = []
        i2 = [np.float64(j) for j in np.array(list(i))]
        for x in range(len(matriz_Ht[0])):
            multiply_ih = i2*matriz_Ht[:,x]
            valid = np.count_nonzero(multiply_ih == 1)
            if valid%2 != 0:
                valor.append(1)
            else:
                valor.append(0)
        sindrome.append(valor)
        if valor == [0, 0, 0]:
            detectable.append("no")
        else:
            detectable.append("si")
    return sindrome, detectable
    

In [256]:
n=7
k=4
p_e = 1e-3
matriz_P = np.array([[1,1,0],[0,1,1],[1,1,1],[1,0,1]], dtype=int)
matriz_Ik = np.identity(k,dtype=int)
matriz_G = np.concatenate((matriz_P.T,matriz_Ik.T),axis=0,dtype=int).T
matriz_H = np.concatenate((matriz_Ik[:-1].T[:-1],matriz_P),axis=0,dtype=int).T
matriz_Ht = matriz_H.T

In [268]:
## calculo de patrones de error
patron_error = np.array(["{0:07b}".format(x) for x in range(128)])
cantidad_error = np.array([x.count('1') for x in patron_error])
sindrome, detectable = calcular_sindrome(patron_error,matriz_Ht)
corregibles = list(map(lambda x: "no" if x>1 else "si", cantidad_error))
probabilidad = [p_e**x * (1-p_e)**(n-x) for x in cantidad_error]

In [258]:
calculo_patrones = pd.DataFrame({'patron_error':patron_error,'cantidad_error':cantidad_error,'sindrome':sindrome,'detectable':detectable,'corregibles':corregibles,'probabilidad':probabilidad})
calculo_patrones

,patron_error,cantidad_error,sindrome,detectable,corregibles,probabilidad
0,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
1,0000001,1,"[1, 0, 1]",si,si,9.940150e-04
2,0000010,1,"[1, 1, 1]",si,si,9.940150e-04
3,0000011,2,"[0, 1, 0]",si,no,9.950100e-07
4,0000100,1,"[0, 1, 1]",si,si,9.940150e-04
...,...,...,...,...,...,...
123,1111011,6,"[0, 1, 1]",si,no,9.990000e-19
124,1111100,5,"[0, 1, 0]",si,no,9.980010e-16
125,1111101,6,"[1, 1, 1]",si,no,9.990000e-19
126,1111110,6,"[1, 0, 1]",si,no,9.990000e-19


In [262]:
###################### Sindrome ######################
#np.where((data['sport'] == '0x0000'), '0', data['sport'])
lista_errores_corregibles = []
lista_sindrome = []
for indice in range(len(calculo_patrones)):
    if (calculo_patrones.patron_error[indice].count('1') < 2):
        lista_errores_corregibles.append(calculo_patrones.patron_error[indice])
        lista_sindrome.append(calculo_patrones.sindrome[indice])
tabla_sindrome = pd.DataFrame({'patron_error_corregible':lista_errores_corregibles,'sindrome':lista_sindrome})
tabla_sindrome

,patron_error_corregible,sindrome
0,0000000,"[0, 0, 0]"
1,0000001,"[1, 0, 1]"
2,0000010,"[1, 1, 1]"
3,0000100,"[0, 1, 1]"
4,0001000,"[1, 1, 0]"
5,0010000,"[0, 0, 1]"
6,0100000,"[0, 1, 0]"
7,1000000,"[1, 0, 0]"


In [ ]:
###################### Transmisor ######################
f = open ('mensaje_codificado.txt','r')
mensaje = f.read()
while(len(mensaje)%4 != 0):
    mensaje+='0'
mensaje = np.array(list(map(list, zip(*[map(int, list(mensaje))] * k))),dtype=int)
mensaje = mensaje @ matriz_G
mensaje_generadora = [[0 if a%2==0 else 1 for a in x] for x in mensaje]
mensaje_enviar = "".join(["".join([str(ele) for ele in sub]) for sub in mensaje_generadora])
mensaje_enviar
f = open ('mensaje_transmisor.txt','w')
f.write(mensaje_enviar)
f.close()

In [271]:
###################### Receptor ######################
f = open ('mensaje_transmisor.txt','r')
mensaje = f.read()
lista_mensaje = []
mensaje_7 = ""
for i in mensaje:
    if len(mensaje_7) < 7:
        mensaje_7 += i
    else:
        lista_mensaje.append(mensaje_7)
        mensaje_7 = ""
cantidad_error_recepcion = np.array([x.count('1') for x in lista_mensaje])
sindrome_recepcion, detectable_recepcion = calcular_sindrome(lista_mensaje,matriz_Ht)
corregibles_recepcion = list(map(lambda x: "no" if x>1 else "si", cantidad_error_recepcion))
probabilidad_recepcion = [p_e**x * (1-p_e)**(n-x) for x in cantidad_error_recepcion]

In [272]:
calculo_patrones_recepcion = pd.DataFrame({'mensaje_recibido':lista_mensaje,'cantidad_error':cantidad_error_recepcion,'sindrome':sindrome_recepcion,'detectable':detectable_recepcion,'corregibles':corregibles_recepcion,'probabilidad':probabilidad_recepcion})
calculo_patrones_recepcion

,mensaje_recibido,cantidad_error,sindrome,detectable,corregibles,probabilidad
0,1110010,4,"[0, 0, 0]",no,no,9.970030e-13
1,0100011,3,"[0, 0, 0]",no,no,9.960060e-10
2,1000100,2,"[1, 1, 1]",si,no,9.950100e-07
3,0000110,2,"[1, 0, 0]",si,no,9.950100e-07
4,1011110,5,"[1, 1, 1]",si,no,9.980010e-16
...,...,...,...,...,...,...
876,1001010,3,"[1, 0, 1]",si,no,9.960060e-10
877,0110111,5,"[0, 1, 0]",si,no,9.980010e-16
878,1111000,4,"[0, 0, 1]",si,no,9.970030e-13
879,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
